<a href="https://colab.research.google.com/github/sak1b0/Thesis/blob/master/lets_get_it_done.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
#import pylab as pl
import pandas as pd
import numpy as np

url='https://raw.githubusercontent.com/sak1b0/Thesis/master/allCSV/letter.csv'

df=pd.read_csv(url,header=None)
#print(df.iloc[0,:])


#----------to handle nominal values---------

my_arr=[]

#track of column indices 
index=0
for item in df.dtypes:
  if(item=='object'):
    #if data type is nominal adding to the array
    my_arr.append(index)
  index=index+1
  
print('object data types: ',len(my_arr))

df=df.values

if(len(my_arr)>0):
  lbl=LabelEncoder()
  for item in my_arr:
    df[:,item] = lbl.fit_transform(df[:,item])
    
  print('after the conversion: ',df[0])    
#--------------------------------------------


df=np.asarray(df)

X, y = np.split(df,[-1],axis=1)
#issue here !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
y=y.astype('float64')

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=101)

#to check additional info 
debug=0

def debug_me():
  print('----------debug start----------')
  print('main data: ',df.shape)
  print('x: ',X.shape)
  print('y: ',y.shape)
  print('x_train: ',x_train.shape)
  print('x_test: ',x_test.shape)
  print('y_train: ',y_train.shape)
  print('y_test: ',y_test.shape)
  print('----------debug end----------')



X=np.asarray(X)
y=np.asarray(y)
# y=y.ravel() #vector to array

x_train=np.asarray(x_train)
x_test=np.asarray(x_test)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

y_train=y_train.ravel()
y_test=y_test.ravel()

debug_me()
#==========preprocessing done============


#==========optimal number of clusters====


no_of_features=x_train.shape[1]
optimal_k=len(np.unique(y_train))
#print('no of features: ',no_of_features)
#print('actual no of classes: ',optimal_k)
#plt.figure(figsize=(6, 4))
#from sklearn.cluster import KMeans
#wcss = []
#for i in range(1, no_of_features):
#    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 101)
#    kmeans.fit(x_train)
#    wcss.append(kmeans.inertia_)
#plt.plot(range(1, no_of_features), wcss)
#plt.title('The Elbow Method')
#plt.xlabel('Number of clusters')
#plt.ylabel('WCSS')
#plt.show()


#=================experiment===========
print("optimal no. of k = ",optimal_k,'\n')

kmeans = KMeans(n_clusters=optimal_k, init = 'k-means++', random_state= 101)

kmeans.fit(x_train)

#print(kmeans.cluster_centers_)  

print('cluster labels: ',kmeans.labels_[0:5,],'\n')

#creating an array as the index to add column to the main df
ind_arr=np.arange(len(x_train))

#adding the index column to the data so that i can understand later
x_train=np.hstack((ind_arr[:, np.newaxis], x_train))

#to see if it has worked and index column is there or not
#print(pd.DataFrame(x_train[:5,]),'\n')
print(x_train[:5,],'\n')

print('-- first column is index --')
#print(pd.DataFrame(x_train[:5,]),'\n')
print(x_train[:5,],'\n')

#print(pd.DataFrame(x_train[:5,1:-1]),'\n')
print(x_train[:5,1:-1],'\n') #without the index here

#mergning y(actual labels with x again)
print(len(x_train))
print(len(y_train))
#np arrays have to be in the proper shape to be appended
y_train = y_train.reshape((y_train.shape[0], 1))
print(x_train.shape)
print(y_train.shape)
x_train=np.append(x_train,y_train, axis=1)
print(x_train[:5,])

print(kmeans.labels_.shape)
cluster_labels = np.asarray(kmeans.labels_)
cluster_labels = cluster_labels.reshape((cluster_labels.shape[0], 1))
print(cluster_labels.shape)

#adding the cluster label column to the data in the last column
train_data=np.append(x_train,cluster_labels, axis=1)
print(train_data[:5])

print('before splitting the length: ',len(train_data))
#splitting portion begins
train_data=np.split(train_data, np.where(np.diff(train_data[:,-1]))[0]+1)
print('after splitting the length: ',len(train_data))

object data types:  1
after the conversion:  [2 4 4 3 2 7 8 2 9 11 7 7 1 8 5 6 25]
----------debug start----------
main data:  (20000, 17)
x:  (20000, 16)
y:  (20000, 1)
x_train:  (14000, 16)
x_test:  (6000, 16)
y_train:  (14000,)
y_test:  (6000,)
----------debug end----------
optimal no. of k =  26 

cluster labels:  [13 14  7 17 10] 

[[0 5 8 8 6 5 5 9 4 6 10 10 9 4 9 4 6]
 [1 4 5 7 4 4 8 7 2 4 9 6 8 7 5 2 7]
 [2 6 9 5 4 3 7 8 6 6 10 5 6 4 7 5 7]
 [3 2 4 3 2 1 7 8 5 6 9 7 10 2 9 4 10]
 [4 3 6 4 4 3 5 10 2 6 10 9 6 1 10 3 6]] 

-- first column is index --
[[0 5 8 8 6 5 5 9 4 6 10 10 9 4 9 4 6]
 [1 4 5 7 4 4 8 7 2 4 9 6 8 7 5 2 7]
 [2 6 9 5 4 3 7 8 6 6 10 5 6 4 7 5 7]
 [3 2 4 3 2 1 7 8 5 6 9 7 10 2 9 4 10]
 [4 3 6 4 4 3 5 10 2 6 10 9 6 1 10 3 6]] 

[[5 8 8 6 5 5 9 4 6 10 10 9 4 9 4]
 [4 5 7 4 4 8 7 2 4 9 6 8 7 5 2]
 [6 9 5 4 3 7 8 6 6 10 5 6 4 7 5]
 [2 4 3 2 1 7 8 5 6 9 7 10 2 9 4]
 [3 6 4 4 3 5 10 2 6 10 9 6 1 10 3]] 

14000
14000
(14000, 17)
(14000, 1)
[[0 5 8 8 6 5 5 9 4 6 10 10 9 4

In [5]:
arr=np.array([[77,77,77,1],[55,55,55,2],[88,88,88,2],[44,44,44,3]])
print(pd.DataFrame(arr))
arr=np.split(arr, np.where(np.diff(arr[:,-1]))[0]+1)
print(pd.DataFrame(arr))
print('no of sub arrays: ',len(arr))
print(arr[1][0])

    0   1   2  3
0  77  77  77  1
1  55  55  55  2
2  88  88  88  2
3  44  44  44  3
                                    0
0                   [[77, 77, 77, 1]]
1  [[55, 55, 55, 2], [88, 88, 88, 2]]
2                   [[44, 44, 44, 3]]
no of sub arrays:  3
[55 55 55  2]
